<a href="https://colab.research.google.com/github/HYUNSOOLEE-6839/TIL/blob/main/Day-30/Make_Development_Environment_For_Image_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import cv2
import os
import glob
import shutil
import random
import string
import numpy as np

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!ls

drive  sample_data


In [7]:
PATH_DEFECT = 'drive/MyDrive/data/Defect_images/'
PATH_MASK = 'drive/MyDrive/data/Mask_images/'
PATH_NODEFECT = 'drive/MyDrive/data/NODefect_images/'

In [8]:
random.seed(0)

defect_list = glob.glob(PATH_DEFECT + '*.png')
mask_list = glob.glob(PATH_MASK + '*.png')
pass_list = glob.glob(PATH_NODEFECT + '*.png')

# Match defect-mask pairs : mask와 defect 데이터를 쌍으로 묶어줌.
new_defect_list = list()
new_mask_list = list()
for defect in defect_list:
  num = defect.split('/')[-1].split('_')[0]
  for mask in mask_list:
    num_mask = mask.split('/')[-1].split('_')[0]
    if num == num_mask:
      new_defect_list.append(defect)
      new_mask_list.append(mask)
      break
defect_list = new_defect_list
mask_list = new_mask_list


In [9]:
# The first dataset given
if os.path.exists('drive/MyDrive/data/1') is False:
  os.mkdir('drive/MyDrive/data/1')
for file_name in pass_list + defect_list:
  if random.randint(0, 9) < 2:
    barcode = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
    shutil.copy(file_name, 'drive/MyDrive/data/1/' + barcode + '.png')

In [11]:
# The second dataset
if os.path.exists('drive/MyDrive/data/2') is False:
  os.mkdir('drive/MyDrive/data/2')
if os.path.exists('drive/MyDrive/data/2/OK') is False:
  os.mkdir('drive/MyDrive/data/2/OK')
if os.path.exists('drive/MyDrive/data/2/FAIL') is False:
  os.mkdir('drive/MyDrive/data/2/FAIL')
idx = 0
for file_name in pass_list:
  img = cv2.imread(file_name)
  height, width, _ = img.shape
  step = height // 2

  for i in range(width // step):
    w = i * step
    if w < width - height and random.randint(0, 9) < 2:
      patch = img[:, w:w+height, :]
      cv2.imwrite('drive/MyDrive/data/2/OK/%04d.png' % idx, patch)
      idx += 1

patch_list = list()
for item in zip(defect_list, mask_list):
  defect, mask = item

  img_d = cv2.imread(defect)
  img_m = cv2.imread(mask)

  height, width, _ = img_d.shape
  step = height // 2
  for i in range(width // step):
    w = i * step
    if w < width - height:
      patch = img_d[:, w:w+height, :]
      patch_d = img_m[:, w:w+height, :]
      if patch_d.sum() > 0:
        patch_list.append(patch)

random.shuffle(patch_list)
patch_list_fraction = patch_list[:len(patch_list)//3]
for idx, patch in enumerate(patch_list_fraction):
  cv2.imwrite('drive/MyDrive/data/2/FAIL/%04d.png' % idx, patch)

In [13]:
# The third dataset
if os.path.exists('drive/MyDrive/data/3') is False:
  os.mkdir('drive/MyDrive/data/3')
if os.path.exists('drive/MyDrive/data/3/OK') is False:
  os.mkdir('drive/MyDrive/data/3/OK')
if os.path.exists('drive/MyDrive/data/3/FAIL') is False:
  os.mkdir('drive/MyDrive/data/3/FAIL')
if os.path.exists('drive/MyDrive/data/3/MASK') is False:
  os.mkdir('drive/MyDrive/data/3/MASK')
idx = 0
for file_name in pass_list:
  img = cv2.imread(file_name)
  height, width, _ = img.shape
  step = height // 2

  for i in range(width // step):
    w = i * step
    if w < width - height and random.randint(0, 9) < 3:
      patch = img[:, w:w+height, :]
      cv2.imwrite('drive/MyDrive/data/3/OK/%04d.png' % idx, patch)
      idx += 1

patch_pair_list = list()
for item in zip(defect_list, mask_list):
  defect, mask = item

  img_d = cv2.imread(defect)
  img_m = cv2.imread(mask)

  height, width, _ = img_d.shape
  step = height // 2
  for i in range(width // step):
    w = i * step
    if w < width - height:
      patch = img_d[:, w:w+height, :]
      patch_d = img_m[:, w:w+height, :]

      if patch_d.sum() > 0:
        patch_pair_list.append((patch, patch_d))

random.shuffle(patch_pair_list)
for idx, pair in enumerate(patch_pair_list):
  patch, patch_d = pair
  cv2.imwrite('drive/MyDrive/data/3/FAIL/%04d.png' % idx, patch)
  cv2.imwrite('drive/MyDrive/data/3/MASK/%04d.png' % idx, patch_d)